In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("hive-queries") \
    .master("spark://spark-master:7077")\
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

# spark.stop()

In [ ]:
#Use WES Database
spark.sql("USE wes")
current_db = spark.sql("SELECT current_database()").collect()[0][0]
print(current_db)

In [ ]:
df_insight_1 = spark.sql("select year,renewables_consumption,fossil_fuel_consumption from wes.world")

df_insight_1.show()

df_insight_1.toPandas().to_csv("insight-1.csv", index=False)


In [ ]:
# Countries that majorly depend on renewables
query_renewables_majority = """
    SELECT country, year, renewables_consumption, fossil_fuel_consumption
    FROM wes.renewable

"""

# Execute the query for renewables-majority countries
df_renewables_majority = spark.sql(query_renewables_majority)

# Show the results for renewables-majority countries
df_renewables_majority.show()

# Save the results for renewables-majority countries to a CSV file
df_renewables_majority.toPandas().to_csv("renewables_majority.csv", index=False)

In [ ]:
# Use the WES database
spark.sql("USE wes")

# Query to get the required data for countries majorly depending on renewables
query_renewables_majority = """
    SELECT country, year, renewables_share_elec, renewables_consumption, renewables_electricity
    FROM wes.renewable
    WHERE renewables_share_elec IS NOT NULL
        AND renewables_share_elec > 0.5
"""

# Execute the query for renewables-majority countries
df_renewables_majority = spark.sql(query_renewables_majority)

# Show the results for renewables-majority countries
df_renewables_majority.show()

# Save the results for renewables-majority countries to a CSV file
df_renewables_majority.toPandas().to_csv("renewables_majority2.csv", index=False)


In [ ]:
# Use the WES database
spark.sql("USE wes")

# Query to get the required data for countries majorly depending on renewables
query_renewables_majority = """
    SELECT country, year, renewables_share_elec, renewables_consumption, renewables_electricity
    FROM wes.world
    WHERE renewables_share_elec IS NOT NULL
        AND renewables_share_elec > 0.5

    ORDER BY renewables_share_elec DESC
"""

# Execute the query for renewables-majority countries
df_renewables_majority = spark.sql(query_renewables_majority)

# Show the results for renewables-majority countries
df_renewables_majority.show()

# Save the results for renewables-majority countries to a CSV file
df_renewables_majority.toPandas().to_csv("renewables_majority4.csv", index=False)

In [ ]:
spark.sql("""CREATE VIEW combined_energy_data AS
SELECT 
    gen.*,
    bio.biofuel_consumption, bio.biofuel_electricity, bio.biofuel_share_elec, bio.biofuel_share_energy,
    coa.coal_consumption, coa.coal_electricity, coa.coal_production, coa.coal_share_elec, coa.coal_share_energy,
    gas.gas_consumption, gas.gas_electricity, gas.gas_production, gas.gas_share_elec, gas.gas_share_energy,
    oil.oil_consumption, oil.oil_electricity, oil.oil_production, oil.oil_share_elec, oil.oil_share_energy,
    foss.fossil_electricity, foss.fossil_fuel_consumption, foss.fossil_share_elec, foss.fossil_share_energy, foss.carbon_intensity_elec,
    ghg.greenhouse_gas_emissions,
    hydr.hydro_consumption, hydr.hydro_electricity, hydr.hydro_share_elec, hydr.hydro_share_energy,
    nuc.nuclear_consumption, nuc.nuclear_electricity, nuc.nuclear_share_elec, nuc.nuclear_share_energy,
    rene.renewables_consumption, rene.renewables_electricity, rene.renewables_share_elec, rene.renewables_share_energy,
    sol.solar_consumption, sol.solar_electricity, sol.solar_share_elec, sol.solar_share_energy,
    win.wind_consumption, win.wind_electricity, win.wind_share_elec, win.wind_share_energy,
    oth.other_renewable_consumption, oth.other_renewable_electricity, oth.other_renewable_exc_biofuel_electricity, oth.other_renewables_share_elec, oth.other_renewables_share_elec_exc_biofuel, oth.other_renewables_share_energy,
    lowc.low_carbon_consumption, lowc.low_carbon_electricity, lowc.low_carbon_share_elec, lowc.low_carbon_share_energy,
    elec_imports.net_elec_imports, elec_imports.net_elec_imports_share_demand
FROM 
    general gen
INNER JOIN biofuel bio ON gen.country = bio.country AND gen.year = bio.year AND gen.iso_code = bio.iso_code
INNER JOIN coal coa ON gen.country = coa.country AND gen.year = coa.year AND gen.iso_code = coa.iso_code
INNER JOIN gas ON gen.country = gas.country AND gen.year = gas.year AND gen.iso_code = gas.iso_code
INNER JOIN oil ON gen.country = oil.country AND gen.year = oil.year AND gen.iso_code = oil.iso_code
INNER JOIN fossil foss ON gen.country = foss.country AND gen.year = foss.year AND gen.iso_code = foss.iso_code
INNER JOIN greenhouse_gas ghg ON gen.country = ghg.country AND gen.year = ghg.year AND gen.iso_code = ghg.iso_code
INNER JOIN hydro hydr ON gen.country = hydr.country AND gen.year = hydr.year AND gen.iso_code = hydr.iso_code
INNER JOIN nuclear nuc ON gen.country = nuc.country AND gen.year = nuc.year AND gen.iso_code = nuc.iso_code
INNER JOIN renewables rene ON gen.country = rene.country AND gen.year = rene.year AND gen.iso_code = rene.iso_code
INNER JOIN solar sol ON gen.country = sol.country AND gen.year = sol.year AND gen.iso_code = sol.iso_code
INNER JOIN wind win ON gen.country = win.country AND gen.year = win.year AND gen.iso_code = win.iso_code
INNER JOIN other_renewables oth ON gen.country = oth.country AND gen.year = oth.year AND gen.iso_code = oth.iso_code
INNER JOIN low_carbon lowc ON gen.country = lowc.country AND gen.year = lowc.year AND gen.iso_code = lowc.iso_code
INNER JOIN electricity_imports elec_imports  ON gen.country = elec_imports.country AND gen.year = elec_imports.year AND gen.iso_code = elec_imports.iso_code;
""")


In [ ]:
spark.sql ("SELECT * from combined_energy_data")

In [ ]:
columnNames = spark.sql("SELECT * FROM combined_energy_data").columns
for columnName in columnNames:
    print(columnName)

In [ ]:
grouped_data_query = """
    SELECT country, year, SUM(renewables_consumption) AS total_renewables_consumption
    FROM combined_energy_data
    GROUP BY country, year
    ORDER BY total_renewables_consumption DESC
"""

grouped_data = spark.sql(grouped_data_query)
grouped_data.show()


In [ ]:
# Assuming 'input_year' is the variable that stores the user input for the year
input_year = 2021  # You can replace this with the actual input from the user

grouped_data_query = f"""
    SELECT country, year, SUM(renewables_consumption) AS total_renewables_consumption
    FROM combined_energy_data
    WHERE year = {input_year}
    GROUP BY country, year
    ORDER BY total_renewables_consumption DESC
"""

grouped_data = spark.sql(grouped_data_query)
grouped_data.show()


In [ ]:
# Assuming 'input_year' is the variable that stores the user input for the year
input_year = 2021  # You can replace this with the actual input from the user

# Using the provided syntax
columnNames = spark.sql("SELECT * FROM combined_energy_data").columns

# Building the query string
non_renewable_query = f"""
    SELECT
      country,
      year,
      fossil_fuel_consumption AS non_renewable_consumption,
      (renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) AS total_consumption,
      fossil_fuel_consumption / (renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) * 100 AS non_renewable_percentage
    FROM
      combined_energy_data
    WHERE
      fossil_fuel_consumption / (renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) * 100 > 50
"""

# Running the query and displaying the results
non_renewable_data = spark.sql(non_renewable_query)
non_renewable_data.show()


In [ ]:
# order of non-renewable
# Assuming 'input_year' is the variable that stores the user input for the year
input_year = 2021  # You can replace this with the actual input from the user

# Using the provided syntax
columnNames = spark.sql("SELECT * FROM combined_energy_data").columns

# Building the query string
non_renewable_query = f"""
    SELECT
      country,
      year,
      SUM(fossil_fuel_consumption) AS total_fossil_fuel_consumption,
      SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) AS total_consumption,
      SUM(fossil_fuel_consumption) / SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) * 100 AS non_renewable_percentage
    FROM
      combined_energy_data
    WHERE
      year = {input_year}
    GROUP BY
      country, year
    HAVING
      non_renewable_percentage > 50
"""

# Running the query and displaying the results
non_renewable_data = spark.sql(non_renewable_query)
non_renewable_data.show()


In [ ]:
# Order of Renewable
# Assuming 'input_year' is the variable that stores the user input for the year
input_year = 2021  # You can replace this with the actual input from the user

# Using the provided syntax
columnNames = spark.sql("SELECT * FROM combined_energy_data").columns

# Building the query string
non_renewable_query = f"""
    SELECT
      country,
      year,
      SUM(fossil_fuel_consumption) AS total_fossil_fuel_consumption,
      SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) AS total_consumption,
      SUM(fossil_fuel_consumption) / SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) * 100 AS non_renewable_percentage
    FROM
      combined_energy_data
    WHERE
      year = {input_year}
    GROUP BY
      country, year
    HAVING
      non_renewable_percentage < 50
"""

# Running the query and displaying the results
non_renewable_data = spark.sql(non_renewable_query)
non_renewable_data.show()


In [ ]:
# Desc order of Total Consumption
# Assuming 'input_year' is the variable that stores the user input for the year
input_year = 2021  # You can replace this with the actual input from the user

# Using the provided syntax
columnNames = spark.sql("SELECT * FROM combined_energy_data").columns

# Building the query string
non_renewable_query = f"""
    SELECT
      country,
      year,
      SUM(fossil_fuel_consumption) AS total_fossil_fuel_consumption,
      SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) AS total_consumption,
      ROUND(SUM(fossil_fuel_consumption) / SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) * 100, 2) AS non_renewable_percentage
    FROM
      combined_energy_data
    WHERE
      year = {input_year}
    GROUP BY
      country, year
    HAVING
      non_renewable_percentage > 50
    ORDER BY
      total_consumption DESC
"""

# Running the query and displaying the results
non_renewable_data = spark.sql(non_renewable_query)
non_renewable_data.show()


In [ ]:
# Assuming 'input_year' is the variable that stores the user input for the year
input_year = 2021  # You can replace this with the actual input from the user

# Using the provided syntax
columnNames = spark.sql("SELECT * FROM combined_energy_data").columns

# Building the query string
non_renewable_query = f"""
    SELECT
      country,
      year,
      SUM(fossil_fuel_consumption) AS total_fossil_fuel_consumption,
      SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) AS total_consumption,
      ROUND(SUM(fossil_fuel_consumption) / SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) * 100, 2) AS non_renewable_percentage,
      ROUND(100 - (SUM(fossil_fuel_consumption) / SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) * 100), 2) AS renewable_percentage
    FROM
      combined_energy_data
    WHERE
      year = {input_year}
    GROUP BY
      country, year
    HAVING
      non_renewable_percentage > 50
    ORDER BY
      total_consumption DESC
"""

# Running the query and displaying the results
non_renewable_data = spark.sql(non_renewable_query)
non_renewable_data.show()


In [ ]:
#Hydro out of renewable
# Assuming 'input_year' is the variable that stores the user input for the year
input_year = 2021  # You can replace this with the actual input from the user

# Using the provided syntax
columnNames = spark.sql("SELECT * FROM combined_energy_data").columns

# Building the query string
non_renewable_query = f"""
    SELECT
      country,
      year,
      SUM(fossil_fuel_consumption) AS total_fossil_fuel_consumption,
      SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) AS total_consumption,
      ROUND(SUM(fossil_fuel_consumption) / SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) * 100, 2) AS non_renewable_percentage,
      ROUND(100 - (SUM(fossil_fuel_consumption) / SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) * 100), 2) AS renewable_percentage,
      ROUND(SUM(hydro_consumption) / (SUM(renewables_consumption) + SUM(other_renewable_consumption)) * 100, 2) AS hydro_percentage
    FROM
      combined_energy_data
    WHERE
      year = {input_year}
    GROUP BY
      country, year
    HAVING
      non_renewable_percentage > 50
    ORDER BY
      total_consumption DESC
"""

# Running the query and displaying the results
non_renewable_data = spark.sql(non_renewable_query)
non_renewable_data.show()


In [ ]:
#Hydro out of total
# Assuming 'input_year' is the variable that stores the user input for the year
input_year = 2021  # You can replace this with the actual input from the user

# Using the provided syntax
columnNames = spark.sql("SELECT * FROM combined_energy_data").columns

# Building the query string
non_renewable_query = f"""
    SELECT
      country,
      year,
      SUM(fossil_fuel_consumption) AS total_fossil_fuel_consumption,
      SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) AS total_consumption,
      ROUND(SUM(fossil_fuel_consumption) / SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) * 100, 2) AS non_renewable_percentage,
      ROUND(100 - (SUM(fossil_fuel_consumption) / SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) * 100), 2) AS renewable_percentage,
      ROUND(SUM(hydro_consumption) / SUM(renewables_consumption + other_renewable_consumption + fossil_fuel_consumption) * 100, 2) AS hydro_percentage
    FROM
      combined_energy_data
    WHERE
      year = {input_year}
    GROUP BY
      country, year
    HAVING
      non_renewable_percentage > 50
    ORDER BY
      total_consumption DESC
"""

# Running the query and displaying the results
non_renewable_data = spark.sql(non_renewable_query)
non_renewable_data.show()


In [ ]:
# Assuming 'input_year' is the variable that stores the user input for the year
input_year = 2021

# Using the provided syntax
columnNames = spark.sql("SELECT * FROM combined_energy_data").columns

# Building the query string for coal
coal_query = f"""
    SELECT
      country,
      year,
      SUM(coal_consumption) AS total_coal_consumption,
      SUM(renewables_consumption + other_renewable_consumption + coal_consumption) AS total_consumption,
      ROUND(SUM(coal_consumption) / SUM(renewables_consumption + other_renewable_consumption + coal_consumption) * 100, 2) AS coal_percentage,
      ROUND(100 - (SUM(coal_consumption) / SUM(renewables_consumption + other_renewable_consumption + coal_consumption) * 100), 2) AS non_coal_percentage,
      ROUND(SUM(hydro_consumption) / SUM(renewables_consumption + other_renewable_consumption + coal_consumption) * 100, 2) AS hydro_percentage
    FROM
      combined_energy_data
    WHERE
      year = {input_year}
    GROUP BY
      country, year
    HAVING
      coal_percentage > 50
    ORDER BY
      total_consumption DESC
"""

# Running the query and displaying the results
coal_data = spark.sql(coal_query)
coal_data.show()
